In [2]:
import numpy as np

In [4]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

# Load the environment
env = gym.make('CartPole-v1')

# Define the neural network
def build_model(input_shape, action_space):
    model = Sequential([
        Dense(24, activation='relu', input_shape=input_shape),
        Dense(24, activation='relu'),
        Dense(action_space, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# Hyperparameters
episodes = 500
gamma = 0.95  # Discount factor
epsilon = 1.0  # Exploration rate
epsilon_decay = 0.995
epsilon_min = 0.01
batch_size = 32
memory = []

# Initialize model
input_shape = (env.observation_space.shape[0],)
action_space = env.action_space.n
model = build_model(input_shape, action_space)

# Training loop
for episode in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, input_shape[0]])
    total_reward = 0
    
    while True:
        # Choose action
        if np.random.rand() <= epsilon:
            action = np.random.choice(action_space)
        else:
            q_values = model.predict(state, verbose=0)
            action = np.argmax(q_values[0])

        # Take action
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, input_shape[0]])
        total_reward += reward
        
        # Store in memory
        memory.append((state, action, reward, next_state, done))
        state = next_state
        
        # Train the model
        if len(memory) > batch_size:
            minibatch = np.random.choice(memory, batch_size, replace=False)
            for s, a, r, ns, d in minibatch:
                target = r if d else r + gamma * np.max(model.predict(ns, verbose=0))
                target_f = model.predict(s, verbose=0)
                target_f[0][a] = target
                model.fit(s, target_f, epochs=1, verbose=0)
        
        if done:
            print(f"Episode: {episode}, Total Reward: {total_reward}")
            break

    # Decay epsilon
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

# Visualize results
plt.plot(rewards)
plt.xlabel('Episodes')
plt.ylabel('Total Reward')
plt.show()


ModuleNotFoundError: No module named 'gym'